# Calcul scientifique en Python 

## Les flottants

In [1]:
0.1 ** 100

1.0000000000000056e-100

In [2]:
3602879701896397 * 10 ** 55 // 2 ** 55

1000000000000000055511151231257827021181583404541015625

In [3]:
.1 + .1 + .1 == .3

False

In [4]:
format(0.1, '.17f')

'0.10000000000000001'

## Numpy

### L'objet `np.ndarray`

Types de données, ordre C et ordre F

### `shape` et indexation

### Création de données

Données pré-déterminées et données aléatoires, grilles

### I/O

## Premier exercice - [Le jeu de la vie de Conway](https://fr.wikipedia.org/wiki/Jeu_de_la_vie)

> Le jeu de la vie n’est pas un jeu, puisqu'il ne nécessite aucun joueur. Il s’agit d’un automate cellulaire, un modèle où chaque état conduit mécaniquement à l’état suivant à partir de règles pré-établies.

> Le « jeu » se déroule sur une grille à deux dimensions, théoriquement infinie (mais de longueur et de largeur finies et plus ou moins grandes dans la pratique), dont les cases — qu’on appelle des « cellules », par analogie avec les cellules vivantes — peuvent prendre deux états distincts : « vivante » ou « morte ».


> À chaque étape, l’évolution d’une cellule est entièrement déterminée par l’état de ses huit voisines de la façon suivante :


* Une cellule morte possédant exactement trois voisines vivantes devient vivante (elle naît).

* Une cellule vivante possédant deux ou trois voisines vivantes le reste, sinon elle meurt.



_Écrivez l'évaluation de la règle pour un pixel_

<button data-toggle="collapse" data-target="#sol" class='btn btn-primary'>Solution</button>
<div id="sol" class="collapse">

```py
import numpy as np
import time

grid_shape = 10, 10
grid = np.random.randint(0, 2, size=grid_shape, dtype=np.uint8)
#grid[1:4, 3] = 1
print(grid)


def eval_pixel(row, col, grid, next_grid):
    """Follows the rules of Conway's game of life to
    flip (or not) a pixel in a grid
    """
    n, m = grid.shape
    sub_grid = grid[
        max(row-1, 0): min(n, row+2),
        max(col-1, 0): min(m, col+2)]
    if grid[row, col] and not (2 < sub_grid.sum() < 5):
        next_grid[row, col] = 0
    elif sub_grid.sum() == 3:
        next_grid[row, col] = 1


def conway_step(grid):
    """Applies Conway's rule for every pixel on a grid
    """
    ii, jj = np.meshgrid(
        np.arange(grid.shape[0]),
        np.arange(grid.shape[1]) )

    next_grid = grid.copy()
    for i, j in zip(ii.ravel(), jj.ravel()):
        eval_neighbors(i, j, grid, next_grid)
    return next_grid


def conway_loop(grid):
    """Play Conways game of life on repeat
    """
    while True:
        try:
            grid = conway_step(grid)
            print(grid)
            time.sleep(1)
            
        except KeyboardInterrupt:
            print('Done')
            break
```

</div>

## Calcul

## Scipy

Exercice 2:

> Créez un signal composé de différentes harmoniques (et de bruit) et évaluez sa FFT.



<button data-toggle="collapse" data-target="#sol2" class='btn btn-primary'>Solution</button>
<div id="sol2" class="collapse">



```python
import numpy as np
import matplotlib.pyplot as plt


t = np.linspace(0, 100, 2**12)
harms = np.random.random(5) * 100
phases = np.random.random(5) * np.pi
signal = np.array(
    [np.sin(omega*t + phi)
      for omega, phi in zip(harms, phases)]
).sum(axis=0)

fft = np.fft.fft(signal)
freqs = np.fft.fftfreq(2**12, 100 / 2**12)

plt.figure()
plt.plot(signal)

plt.figure()
plt.plot(np.abs(freqs), np.abs(fft))

print(np.sort(harms) / (2*np.pi))
```
</div>

### Exercice 3:

> Convoluez le précédent signal par un filtre passe bas.

<button data-toggle="collapse" data-target="#sol3" class='btn btn-primary'>Solution</button>
<div id="sol3" class="collapse">


```python
from scipy.signal import fftconvolve

filtr = np.ones(100)
filtr[10:] = 0

convd = fftconvolve(signal, filtr, mode='same')

plt.plot(t, signal)
plt.plot(t, convd, alpha=0.4)
```

## Pandas 

#### I/O

#### Exercice:

Get data from a website and create a DataFrame.

Go to https://openweathermap.org/api/hourly-forecast

<button data-toggle="collapse" data-target="#sol4" class='btn btn-primary'>Solution</button>

<div id="sol4" class="collapse">

```python
import requests
import json

response = requests.get(
    "http://samples.openweathermap.org/data/2.5/forecast/hourly?id=524901&appid=b6907d289e10d714a6e88b30761fae22"
)

whole = pd.DataFrame.from_dict(json.loads(response.text)['list'])
main = pd.DataFrame.from_records(whole['main'])
main.set_index(pd.DatetimeIndex(whole["dt_txt"]), inplace=True)
```

</div>

### Indexation et sélection

**Exercice:**

Récupérer les jours pour lesquels la température est inférieure à 0°C

### Split / apply / combine

**Exercice:**

Utiliser `np.cut` pour créer des gammes de températures, trouver la moyenne de chaque valeur par gamme.

<button data-toggle="collapse" data-target="#sol5" class='btn btn-primary'>Solution</button>
<div id="sol5" class="collapse">

```python
temp_cats = pd.cut(main['temp'], 5)
main['temp_cat'] = temp_cats
main.groupby('temp_cat').mean()
```

</div>